# Natural Language Processing

1. What is NLP?
    * A variety of areas in which linguistics meets mathematics and/or computer science
    * Example of interacting with a speech bot:
        - Speech-to-text: creating letters/words/sentences from continuous sounds
        - Natural language understanding: mapping the sentences to some semantic representation
        - Dialog systems: determining appropriate response given semantic representation
        - Information extraction/retrieval: collecting any data needed to make the response
        - Natural language generation: generating words and sentences in a target language
        - Text-to-speech: creating continuous sounds from letters/words/sentences
    * Locally, we tend to do *Information Extraction*, which is essentially converting free-text to structured data
2. Terminology
    * Corpus (pl. corpora): dataset, consisting of documents/notes
    * Document/note: the smallest self-contained unit of analysis (e.g., a comment, post, blog, newspaper article, clinical note, etc.)
    * Token: smallest word or punctuation (_don't_ can have two tokens: _do_ and _n't_)
    * Lemma/stem: canonical form of word (map _hurt_ and _hurts_ to the same word)
2. Sentiment analysis example
    * Based on terms, can we learn about people's attitudes, emotions, or perceptions
3. Spam/ham classification
    * Classic example using ML in NLP
4. nltk
    * A popular library, but also very useful to _learn_ basic NLP
    * http://www.nltk.org/book/
    * POS-tagging
    * Syntactic analysis
    * Spell-correction/edit distance

## Sentiment Analysis

Sentiment analysis is concerned about learning the subjective information about actors in a particular setting. This is often concerned with detecting 'positive' vs 'neutral' vs 'negative' comments, though we will consider an example of the use of male and female terms in a particular document. 

In [25]:
import nltk
# sentence splitting/tokenization, etc require models, and these must be downloaded separately
nltk.download()  # or download from http://www.nltk.org/nltk_data/

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [118]:
stopwords = nltk.corpus.stopwords.words('english')

In [119]:
from enum import Enum

class Category(Enum):
    C1 = 0
    C2 = 1
    BOTH = 2
    UNKNOWN = 3

In [120]:
MALE_TERMS = {
    'man', 'men', 'mans', 'mens', 'guy', 'guys', 'him', 'he', 'hes', 'his',
    'boy', 'boys', 'boyfriend', 'boyfriends', 'brother', 'brothers', 'dad', 'dads',
    'father', 'fathers', 'spokesman', 'spokesmen', 'spokesmans', 'gentleman',
    'gentlemans', 'gentlemen', 'gentlemens', 'groom', 'grooms', 'son', 'sons', 
    'grandson', 'grandsons', 'grandpa', 'grandpas', 'grandfather', 'grandfathers',
    'fiance', 'fiances', 'male', 'males', 'king', 'kings', 'mr', 'himself',
    'sir', 'sirs', 'nephew', 'nephews', 'uncle', 'uncles', 'husband', 'husbands',
    'waiter', 'waiters', 'widower', 'widowers', 'prince', 'princes', 'priest',
    'priests', 'chairman', 'chairmen', 'chairmans', 'chairmens'
}
FEMALE_TERMS = {
    'woman', 'women', 'womans', 'womens', 'gal', 'gals', 'her', 'she', 'shes', 'hers',
    'girl', 'girls', 'girlfriend', 'girlfriends', 'sister', 'sisters', 'mom', 'moms',
    'mother', 'mothers', 'spokeswoman', 'spokeswomen', 'spokeswomans', 'lady', 'ladys'
    'ladies', 'bride', 'brides', 'daughter', 'daughters', 'miss',
    'granddaughter', 'granddaughters', 'grandma', 'grandmas', 'grandmother', 'grandmothers',
    'fiancee', 'fiancees', 'female', 'females', 'queen', 'queens', 'mrs', 'herself',
    'niece', 'nieces', 'aunt', 'aunt', 'wife', 'wifes', 'wives',
    'waitress', 'waitresses', 'widow', 'widows', 'princess', 'princesses', 'priestess',
    'priestesses', 'chairwoman', 'chairwomans', 'chairwomen', 'chairwomens'
}

In [126]:
# remove punctuation
from string import punctuation
TABLE = str.maketrans('', '', punctuation + '’—“')

In [122]:
def get_sentence(text):
    for sent in nltk.sent_tokenize(text):
        yield {word.lower().translate(TABLE) for word in nltk.word_tokenize(sent) if word.translate(TABLE).strip()}
            
def parse_category(texts, cat1, cat2, label1, label2):
    all_sents = Counter()
    all_words = Counter()
    terms = defaultdict(lambda: Counter())
    for title, text in texts:
        sents, words = count_category(get_sentence(text), cat1, cat2, terms)
        all_sents += sents
        all_words += words
        total = sum(words.values())
        for cat, count in words.items():
            percent = count * 100 / total
            num_sents = sents[cat]
            if cat == Category.C1:
                label = label1
            elif cat == Category.C2:
                label = label2
            else:
                label = cat
            print(title)
            print(f'{percent:0.2f}% {label} ({num_sents} sentences)')
    for cat in terms:
        if cat == Category.C1:
            label = label1
        elif cat == Category.C2:
            label = label2
        else:
            label = cat
        print(f'{label}\n\t', end='')
        for term in terms[cat].most_common(20):
            print(f'{term[0]}', end=',')
        print()

In [123]:
from collections import Counter, defaultdict

def count_category(sentences, cat1, cat2, terms):
    sents = Counter()
    words = Counter()
    for sent in sentences:
        cat = judge_category(sent, cat1, cat2)
        sents[cat] += 1  # number of sentences with this category
        words[cat] += len(sent)  # number of words in this category
        terms[cat].update({s for s in sent if s not in stopwords})  # word counts in this category
    return words, sents

In [124]:
def judge_category(words, cat1, cat2):
    count1 = len(cat1 & words)
    count2 = len(cat2 & words)
    if count1 > 0 and count2 > 0:
        return Category.BOTH
    elif count1 > 0:
        return Category.C1
    elif count2 > 0:
        return Category.C2
    else:
        return Category.UNKNOWN

In [127]:
texts = []
with open(r'data\grandmothers-love.txt') as fh:
    texts.append(('grandmothers-love', fh.read()))

parse_category(texts, MALE_TERMS, FEMALE_TERMS, 'male', 'female')

grandmothers-love
6.67% male (98 sentences)
grandmothers-love
35.56% female (468 sentences)
grandmothers-love
55.56% Category.UNKNOWN (457 sentences)
grandmothers-love
2.22% Category.BOTH (7 sentences)
male
	community,history,beason,shaped,views,foundation,face,socialjustice,today,tyrone,times,issues,serves,seattle,columnist,new,parents,early,card,attended,
female
	knew,woman,teacher,used,black,way,back,name,grandmother,white,mama,big,might,day,first,said,two,pride,told,house,
Category.UNKNOWN
	people,justice,history,eyes,turn,teacher,mean,harm,life,better,living,born,country,keep,toward,see,opportunity,time,write,like,
Category.BOTH
	big,dad,mama,mother,


In [136]:
docs = iter((fileid, nltk.corpus.reuters.raw(fileid)) for fileid in nltk.corpus.reuters.fileids())
parse_category(docs, MALE_TERMS, FEMALE_TERMS, 'male', 'female')

test/14826
83.87% Category.UNKNOWN (565 sentences)
test/14826
16.13% male (99 sentences)
test/14828
100.00% Category.UNKNOWN (92 sentences)
test/14829
100.00% Category.UNKNOWN (150 sentences)
test/14832
100.00% Category.UNKNOWN (132 sentences)
test/14833
100.00% Category.UNKNOWN (144 sentences)
test/14839
100.00% Category.UNKNOWN (179 sentences)
test/14840
26.32% male (131 sentences)
test/14840
73.68% Category.UNKNOWN (230 sentences)
test/14841
100.00% Category.UNKNOWN (48 sentences)
test/14842
100.00% Category.UNKNOWN (106 sentences)
test/14843
82.86% Category.UNKNOWN (504 sentences)
test/14843
17.14% male (124 sentences)
test/14844
60.00% Category.UNKNOWN (56 sentences)
test/14844
40.00% male (37 sentences)
test/14849
33.33% male (79 sentences)
test/14849
66.67% Category.UNKNOWN (141 sentences)
test/14852
100.00% Category.UNKNOWN (236 sentences)
test/14854
100.00% Category.UNKNOWN (86 sentences)
test/14858
86.36% Category.UNKNOWN (349 sentences)
test/14858
13.64% male (68 sentences)


66.67% Category.UNKNOWN (367 sentences)
test/15149
23.33% male (117 sentences)
test/15149
10.00% female (40 sentences)
test/15152
100.00% Category.UNKNOWN (20 sentences)
test/15153
100.00% Category.UNKNOWN (18 sentences)
test/15154
56.25% Category.UNKNOWN (218 sentences)
test/15154
43.75% male (151 sentences)
test/15156
100.00% Category.UNKNOWN (127 sentences)
test/15157
100.00% Category.UNKNOWN (18 sentences)
test/15161
66.67% Category.UNKNOWN (59 sentences)
test/15161
33.33% male (15 sentences)
test/15162
100.00% Category.UNKNOWN (81 sentences)
test/15171
60.00% male (72 sentences)
test/15171
40.00% Category.UNKNOWN (26 sentences)
test/15172
100.00% Category.UNKNOWN (12 sentences)
test/15175
100.00% Category.UNKNOWN (24 sentences)
test/15179
100.00% male (116 sentences)
test/15180
100.00% Category.UNKNOWN (30 sentences)
test/15185
100.00% Category.UNKNOWN (9 sentences)
test/15188
100.00% Category.UNKNOWN (28 sentences)
test/15189
100.00% Category.UNKNOWN (101 sentences)
test/15190
10

20.00% male (18 sentences)
test/15547
100.00% Category.UNKNOWN (69 sentences)
test/15548
100.00% Category.UNKNOWN (22 sentences)
test/15549
93.33% Category.UNKNOWN (703 sentences)
test/15549
6.67% male (45 sentences)
test/15550
100.00% Category.UNKNOWN (62 sentences)
test/15551
100.00% Category.UNKNOWN (418 sentences)
test/15552
75.00% Category.UNKNOWN (215 sentences)
test/15552
25.00% male (65 sentences)
test/15553
100.00% Category.UNKNOWN (21 sentences)
test/15556
100.00% Category.UNKNOWN (565 sentences)
test/15558
100.00% Category.UNKNOWN (50 sentences)
test/15559
100.00% Category.UNKNOWN (25 sentences)
test/15560
100.00% Category.UNKNOWN (38 sentences)
test/15561
100.00% Category.UNKNOWN (43 sentences)
test/15562
66.67% male (45 sentences)
test/15562
33.33% Category.UNKNOWN (25 sentences)
test/15563
100.00% Category.UNKNOWN (34 sentences)
test/15565
100.00% Category.UNKNOWN (205 sentences)
test/15566
100.00% Category.UNKNOWN (10 sentences)
test/15567
100.00% Category.UNKNOWN (254 s

test/15894
12.50% male (25 sentences)
test/15895
100.00% Category.UNKNOWN (24 sentences)
test/15896
100.00% Category.UNKNOWN (33 sentences)
test/15897
100.00% Category.UNKNOWN (20 sentences)
test/15898
100.00% Category.UNKNOWN (37 sentences)
test/15899
100.00% Category.UNKNOWN (40 sentences)
test/15900
100.00% Category.UNKNOWN (52 sentences)
test/15901
100.00% Category.UNKNOWN (19 sentences)
test/15902
100.00% Category.UNKNOWN (24 sentences)
test/15903
100.00% Category.UNKNOWN (24 sentences)
test/15904
100.00% Category.UNKNOWN (24 sentences)
test/15906
100.00% Category.UNKNOWN (476 sentences)
test/15908
100.00% Category.UNKNOWN (38 sentences)
test/15909
100.00% Category.UNKNOWN (48 sentences)
test/15910
100.00% Category.UNKNOWN (148 sentences)
test/15911
100.00% Category.UNKNOWN (96 sentences)
test/15912
100.00% Category.UNKNOWN (102 sentences)
test/15913
75.00% male (64 sentences)
test/15913
25.00% Category.UNKNOWN (22 sentences)
test/15914
100.00% Category.UNKNOWN (186 sentences)
tes

test/16233
85.71% Category.UNKNOWN (103 sentences)
test/16233
14.29% male (26 sentences)
test/16234
100.00% Category.UNKNOWN (37 sentences)
test/16236
100.00% Category.UNKNOWN (69 sentences)
test/16238
100.00% Category.UNKNOWN (294 sentences)
test/16241
100.00% Category.UNKNOWN (152 sentences)
test/16243
100.00% Category.UNKNOWN (30 sentences)
test/16244
100.00% Category.UNKNOWN (34 sentences)
test/16246
100.00% Category.UNKNOWN (51 sentences)
test/16247
100.00% Category.UNKNOWN (50 sentences)
test/16248
87.50% Category.UNKNOWN (139 sentences)
test/16248
12.50% male (21 sentences)
test/16250
100.00% Category.UNKNOWN (100 sentences)
test/16251
100.00% Category.UNKNOWN (83 sentences)
test/16252
66.67% Category.UNKNOWN (94 sentences)
test/16252
33.33% male (56 sentences)
test/16255
100.00% Category.UNKNOWN (47 sentences)
test/16256
100.00% Category.UNKNOWN (178 sentences)
test/16257
100.00% Category.UNKNOWN (231 sentences)
test/16258
100.00% Category.UNKNOWN (75 sentences)
test/16260
100.

...

training/7628
95.83% Category.UNKNOWN (487 sentences)
training/7628
4.17% male (20 sentences)
training/7629
100.00% Category.UNKNOWN (241 sentences)
training/7631
100.00% Category.UNKNOWN (32 sentences)
training/7632
69.23% Category.UNKNOWN (160 sentences)
training/7632
30.77% male (91 sentences)
training/7633
42.86% male (69 sentences)
training/7633
57.14% Category.UNKNOWN (77 sentences)
training/7634
100.00% Category.UNKNOWN (39 sentences)
training/7635
100.00% Category.UNKNOWN (142 sentences)
training/7636
100.00% Category.UNKNOWN (28 sentences)
training/7637
100.00% Category.UNKNOWN (11 sentences)
training/7638
100.00% Category.UNKNOWN (11 sentences)
training/7639
100.00% Category.UNKNOWN (92 sentences)
training/764
100.00% Category.UNKNOWN (39 sentences)
training/7640
33.33% male (30 sentences)
training/7640
66.67% Category.UNKNOWN (49 sentences)
training/7641
100.00% Category.UNKNOWN (12 sentences)
training/7642
100.00% Category.UNKNOWN (157 sentences)
training/7643
85.71% Catego

training/8109
41.67% male (122 sentences)
training/811
100.00% Category.UNKNOWN (39 sentences)
training/8111
100.00% Category.UNKNOWN (165 sentences)
training/8112
100.00% Category.UNKNOWN (39 sentences)
training/8113
62.50% Category.UNKNOWN (107 sentences)
training/8113
37.50% male (68 sentences)
training/8115
96.55% Category.UNKNOWN (570 sentences)
training/8115
3.45% male (27 sentences)
training/8117
76.92% male (226 sentences)
training/8117
23.08% Category.UNKNOWN (90 sentences)
training/8119
66.67% Category.UNKNOWN (52 sentences)
training/8119
33.33% male (24 sentences)
training/8120
100.00% Category.UNKNOWN (14 sentences)
training/8123
100.00% Category.UNKNOWN (14 sentences)
training/8125
100.00% Category.UNKNOWN (70 sentences)
training/8126
100.00% Category.UNKNOWN (50 sentences)
training/8130
91.67% Category.UNKNOWN (237 sentences)
training/8130
8.33% male (22 sentences)
training/8131
30.00% Category.UNKNOWN (80 sentences)
training/8131
70.00% male (163 sentences)
training/8132

60.00% Category.UNKNOWN (165 sentences)
training/8597
40.00% male (154 sentences)
training/8598
72.73% Category.UNKNOWN (226 sentences)
training/8598
27.27% male (93 sentences)
training/8599
50.00% Category.UNKNOWN (116 sentences)
training/8599
50.00% male (123 sentences)
training/86
100.00% Category.UNKNOWN (17 sentences)
training/8600
58.33% Category.UNKNOWN (139 sentences)
training/8600
41.67% male (113 sentences)
training/8602
71.43% Category.UNKNOWN (155 sentences)
training/8602
28.57% male (59 sentences)
training/8603
75.00% Category.UNKNOWN (79 sentences)
training/8603
25.00% male (23 sentences)
training/8604
100.00% Category.UNKNOWN (132 sentences)
training/8605
100.00% Category.UNKNOWN (9 sentences)
training/8606
70.00% Category.UNKNOWN (177 sentences)
training/8606
30.00% male (66 sentences)
training/8607
85.71% Category.UNKNOWN (138 sentences)
training/8607
14.29% male (17 sentences)
training/8608
44.44% male (101 sentences)
training/8608
55.56% Category.UNKNOWN (104 sentenc

training/9030
100.00% Category.UNKNOWN (62 sentences)
training/9031
100.00% Category.UNKNOWN (100 sentences)
training/9032
100.00% Category.UNKNOWN (300 sentences)
training/9033
100.00% Category.UNKNOWN (68 sentences)
training/9034
100.00% Category.UNKNOWN (46 sentences)
training/9036
100.00% Category.UNKNOWN (156 sentences)
training/9039
100.00% Category.UNKNOWN (13 sentences)
training/904
100.00% Category.UNKNOWN (61 sentences)
training/9040
100.00% Category.UNKNOWN (29 sentences)
training/9041
100.00% Category.UNKNOWN (9 sentences)
training/9044
100.00% Category.UNKNOWN (21 sentences)
training/9045
100.00% Category.UNKNOWN (56 sentences)
training/9047
78.26% Category.UNKNOWN (341 sentences)
training/9047
21.74% male (100 sentences)
training/9048
57.14% Category.UNKNOWN (83 sentences)
training/9048
42.86% male (93 sentences)
training/9049
100.00% Category.UNKNOWN (28 sentences)
training/9051
100.00% Category.UNKNOWN (114 sentences)
training/9053
100.00% Category.UNKNOWN (60 sentences

37.50% Category.UNKNOWN (33 sentences)
training/9425
100.00% Category.UNKNOWN (31 sentences)
training/9426
100.00% Category.UNKNOWN (10 sentences)
training/9427
100.00% Category.UNKNOWN (51 sentences)
training/9428
100.00% Category.UNKNOWN (20 sentences)
training/9429
100.00% Category.UNKNOWN (47 sentences)
training/943
100.00% Category.UNKNOWN (25 sentences)
training/9431
100.00% Category.UNKNOWN (43 sentences)
training/9432
100.00% Category.UNKNOWN (68 sentences)
training/9433
100.00% Category.UNKNOWN (91 sentences)
training/9434
100.00% Category.UNKNOWN (63 sentences)
training/9435
100.00% Category.UNKNOWN (10 sentences)
training/9436
60.00% Category.UNKNOWN (239 sentences)
training/9436
40.00% male (288 sentences)
training/9437
100.00% Category.UNKNOWN (180 sentences)
training/9438
100.00% Category.UNKNOWN (26 sentences)
training/944
100.00% Category.UNKNOWN (27 sentences)
training/9441
100.00% Category.UNKNOWN (33 sentences)
training/9443
100.00% Category.UNKNOWN (123 sentences)
t